In [ ]:
import numpy as np
from scipy.stats import norm
import scipy.stats
import scipy
import math
from matplotlib import pyplot as plt

In [ ]:
def reg(x): # piecewise constant regression function (other piecewise constant functions will 
            # also be considered later, denoted by regnew1, regnew2 ,etc)
    if 0<x<1:
        return 1
    if 1<=x<2:
        return 2

In [ ]:
def eta(x,y,eps): # the kernel function eta (other types of kernels will also be considered later)
    if np.abs(x-y)<=eps:
        return 1
    if np.abs(x-y)>eps:
        return 0

In [ ]:
def Lap(n,X): # the unnormalized Laplacian matrix
    W=np.zeros((n,n))
    D=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            W[i,j]=eta(X[i],X[j],eps)
        D[i,i]=np.sum(W[i,:])
    return (D-W)/(n*eps**3)

In [ ]:
# MSE vs the sample size n
# listn2 contains the sample sizes n considered
# b indexes the repetitions and the results are averaged over 200 runs

listn2=[500,600,700,800,900,1000]
l2err=[]
for n in listn2:
    l2errb=np.zeros((200,1))
    for b in range(200):
        X=np.random.uniform(0,2,n)  # uniform design (later we will also consider other random designa including truncated normal design, etc.)
        error=np.random.normal(0, 1, size=n)
        eps=n**(-(1/2-0.001))
        K=math.floor(n**(1/2-0.001))
        eigVal,eigVectors = scipy.sparse.linalg.eigsh(-Lap(n,X), K,  which='LA') # calculate eigendecomposition

        Vk=np.zeros((n,K))
        for j in range(K):
            Vk[:,j]=eigVectors[:,j]
    
        Y=np.zeros((n,1))
        FX=np.zeros((n,1))
        for i in range(n):
            Y[i]=reg(X[i])+error[i]
            FX[i]=reg(X[i])
        predict=Vk@Vk.T@Y
        l2errb[b]=np.linalg.norm(FX-predict, axis=0)/n
    l2err.append(np.mean(l2errb))

In [ ]:
# calculating the line with the theoretical slope to match the intercept

b=np.log(l2err[0])-(-1/2)*np.log(500)
theb=(-1/2)*np.log(listn2)+b

In [ ]:
# plot: MSE vs n

plt.scatter(np.log(listn),np.log(l2err))
plt.plot(np.log(listn),np.log(l2err),"-b", label="PCR-FLE")
plt.plot(np.log(listn),theb,"-r", label="theoretical slope = -1/2")
plt.xlabel("Sample size")
plt.ylabel("Mean squre error")
plt.legend(loc="upper right")
plt.title('d=1, piecewise constant function')
plt.show()

In [ ]:
def reg2(x): # piecewise polynoimal regression function (other piecewise constant regression functions 
             # will also be considered later denoted by regnew1, regnew2 ,etc)
    if 0<x<=1:
        return x
    if 1<x<=1.5:
        return x**2
    if 1.5<x<2:
        return (x+x**2)

In [ ]:
# MSE vs the sample size n
# listn2 contains the sample sizes n considered
# b indexes the repetitions and the results are averaged over 200 runs

listn2=[500,600,700,800,900,1000]
l2err2=[]
for n in listn2:
    l2errb2=np.zeros((200,1))
    for b in range(200):
        #X=np.random.uniform(0,2,n)
        X=scipy.stats.truncnorm.rvs(0, 2, size = n) # truncated normal design
        error=np.random.normal(0, 1, size=n)
        eps=n**(-(1/2-0.001))
        K=math.floor(n**(1/2-0.001))
        eigVal,eigVectors = scipy.sparse.linalg.eigsh(-Lap(n,X), K,  which='LA') # calculate eigendecomposition

        Vk=np.zeros((n,K))
        for j in range(K):
            Vk[:,j]=eigVectors[:,j]
    
        Y=np.zeros((n,1))
        FX=np.zeros((n,1))
        for i in range(n):
            Y[i]=reg2(X[i])+error[i]
            FX[i]=reg2(X[i])
        predict=Vk@Vk.T@Y
        l2errb2[b]=np.linalg.norm(FX-predict, axis=0)/n
    l2err2.append(np.mean(l2errb2))

In [ ]:
# calculating the line with the theoretical slope to match the intercept


b2=np.log(l2err2[0])-(-1/2)*np.log(500)
theb2=(-1/2)*np.log(listn)+b2

In [ ]:
plt.scatter(np.log(listn),np.log(l2err2))
plt.plot(np.log(listn),np.log(l2err2),"-b", label="PCR-FLE")
plt.plot(np.log(listn),theb2,"-r", label="theoretical slope = -1/2")
plt.xlabel("Sample size")
plt.ylabel("Mean squre error")
plt.legend(loc="upper right")
plt.title('d=1, piecewise polynomial function')
plt.show()

In [ ]:
# This part involves the grid search of the tunning parameters

# 1) grid search of K: the number of eigenvalues used

n=100
l2err3=[]
for K in range(0,99,2):
    l2errb3=np.zeros((200,1))
    #X=scipy.stats.truncnorm.rvs(0, 2, size = n) 
    for b in range(200):
        #X=np.random.uniform(0,2,n)
        X=scipy.stats.truncnorm.rvs(0, 2, size = n) 
        error=np.random.normal(0, 1, size=n)
        eps=n**(-(1/2-0.001))
        eigVal,eigVectors = scipy.sparse.linalg.eigsh(-Lap(n,X), K+1,  which='LA')

        Vk=np.zeros((n,K))
        for j in range(K):
            Vk[:,j]=eigVectors[:,j]
    
        Y=np.zeros((n,1))
        FX=np.zeros((n,1))
        for i in range(n):
            Y[i]=reg2(X[i])+error[i]
            FX[i]=reg2(X[i])
        predict=Vk@Vk.T@Y
        l2errb3[b]=np.linalg.norm(FX-predict, axis=0)/n
    l2err3.append(np.mean(l2errb3))

In [ ]:
plt.scatter(range(0,99,2),l2err3)
plt.plot(range(0,99,2),l2err3,"-b")
plt.xlabel("Number of eigenvectors K")
plt.ylabel("Mean squre error")
plt.title('d=1, piecewise polynomial function')
plt.show()

In [ ]:
# This part involves the grid search of the tunning parameters

# 2) grid search of epsilon: the graph radius

n=100
l2err4=[]
for N in range(25,499,2):
    l2errb4=np.zeros((200,1))
    #X=scipy.stats.truncnorm.rvs(0, 2, size = n)
    for b in range(200):
        X=scipy.stats.truncnorm.rvs(0, 2, size = n)
        error=np.random.normal(0, 1, size=n)
        eps=N/500
        K=10
        eigVal,eigVectors = scipy.sparse.linalg.eigsh(-Lap(n,X), K+1,  which='LA')

        Vk=np.zeros((n,K))
        for j in range(K):
            #Vk[:,j]=eigVectors[:,j]/np.sqrt((np.linalg.norm(eigVectors[:,j], axis=0))**2/n)
            Vk[:,j]=eigVectors[:,j]
    
        Y=np.zeros((n,1))
        FX=np.zeros((n,1))
        for i in range(n):
            Y[i]=reg2(X[i])+error[i]
            FX[i]=reg2(X[i])
        predict=Vk@Vk.T@Y
        l2errb4[b]=np.linalg.norm(FX-predict, axis=0)/n
    l2err4.append(np.mean(l2errb4))

In [ ]:
plt.scatter(range(10,99,2),l2err4)
plt.plot(range(10,99,2),l2err4,"-b")
plt.xlabel("Number of eigenvectors K")
plt.ylabel("Mean squre error")
plt.title('d=1, piecewise polynomial function')
plt.show()

In [ ]:
a=[i/500 for i in range(25,499,2)]
plt.scatter(a,l2err4)
plt.plot(a,l2err4,"-b")
plt.xlabel("Graph radius")
plt.ylabel("Mean squre error")
plt.title('d=1, piecewise polynomial function')
plt.show()

In [ ]:
def regnew1(x): # piecewise constant regression function
    if 0<x<1:
        return 1
    if 1<=x<2:
        return 0.5
    if 2<=x<3:
        return 2
    if 3<=x<5:
        return -2.5

In [ ]:
def regnew2(x): # piecewise polynomial regression function
    if 0<x<1:
        return x
    if 1<=x<2:
        return 2*x**2+2
    if 2<=x<3:
        return -x+2
    if 3<=x<5:
        return 0.2*x**3-2*x-4

In [ ]:
# Plotting of of fitted regression curve (using regnew1 as true regression fucntion)

# listn: the sample size 
# eps and K are the tuning parameters optimized by double grid search
# averpredict records the estimated value after 200 repetions

listn=[1000]
eps=beps
K=bK
for n in listn:
    X=np.random.uniform(0,5,n)
    averpredict=np.matrix(np.zeros((n,200)))
    for b in range(200):        
        error=np.random.normal(0, 1, size=n)
        #eps=n**(-(1/2-0.001))
        #K=math.floor(n**(1/2-0.001))
        eigVal,eigVectors = scipy.sparse.linalg.eigsh(-Lap(n,X), K,  which='LA')

        Vk=np.zeros((n,K))
        for j in range(K):
            Vk[:,j]=eigVectors[:,j]
    
        Y=np.zeros((n,1))
        FX=np.zeros((n,1))
        for i in range(n):
            Y[i]=regnew1(X[i])+error[i]
            FX[i]=regnew1(X[i])
        predict=Vk@Vk.T@Y
        averpredict[:,b]=predict

In [ ]:
# averpredict records the estimated value after 200 repetions
# xx used to plot the true regression function regnew1

plt.scatter(X,np.array(np.mean(averpredict,axis=1)),label="PCR-FLE")
xx = np.linspace(0, 5, 1000)
plt.plot(xx,[regnew1(xxx) for xxx in xx],"-r",label="True regression function")
plt.xlabel("x")
plt.title('d=1, piecewise constant function')
plt.legend(loc="best")
plt.show()

In [ ]:
# Plotting of fitted regression curve (using regnew2 as true regression function)

# listn: the sample size 
# eps and K are the tuning parameters optimized by double grid search
# averpredict records the estimated value after 200 repetions

listn=[1000]
eps=beps
K=bK
for n in listn:
    X=np.random.uniform(0,5,n)
    averpredict=np.matrix(np.zeros((n,200)))
    for b in range(200):        
        error=np.random.normal(0, 1, size=n)
        #eps=n**(-(1/2-0.001))
        #K=math.floor(n**(1/2-0.001))
        eigVal,eigVectors = scipy.sparse.linalg.eigsh(-Lap(n,X), K,  which='LA')

        Vk=np.zeros((n,K))
        for j in range(K):
            Vk[:,j]=eigVectors[:,j]
    
        Y=np.zeros((n,1))
        FX=np.zeros((n,1))
        for i in range(n):
            Y[i]=regnew2(X[i])+error[i]
            FX[i]=regnew2(X[i])
        predict=Vk@Vk.T@Y
        averpredict[:,b]=predict

In [ ]:
# averpredict records the estimated value after 200 repetions
# xx used to plot the true regression function regnew2

plt.scatter(X,np.array(np.mean(averpredict,axis=1)),label="PCR-FLE")
xx = np.linspace(0, 5, 1000)
plt.plot(xx,[regnew2(xxx) for xxx in xx],"-r",label="True regression function")
plt.xlabel("x")
plt.title('d=1, piecewise polynomial function')
plt.legend(loc="best")
plt.show()

In [ ]:
# The following part again uses regnew1 and changes the sample size to n=100 and plots the fitted regression curve

listn=[100]
eps=beps
K=bK
for n in listn:
    X=np.random.uniform(0,5,n)
    averpredict=np.matrix(np.zeros((n,200)))
    for b in range(200):        
        error=np.random.normal(0, 1, size=n)
        eigVal,eigVectors = scipy.sparse.linalg.eigsh(-Lap(n,X), K,  which='LA')

        Vk=np.zeros((n,K))
        for j in range(K):
            #Vk[:,j]=eigVectors[:,j]/np.sqrt((np.linalg.norm(eigVectors[:,j], axis=0))**2/n)
            Vk[:,j]=eigVectors[:,j]
    
        Y=np.zeros((n,1))
        FX=np.zeros((n,1))
        for i in range(n):
            Y[i]=regnew1(X[i])+error[i]
            FX[i]=regnew1(X[i])
        predict=Vk@Vk.T@Y
        averpredict[:,b]=predict

In [ ]:
plt.scatter(X,np.array(np.mean(averpredict,axis=1)),label="PCR-FLE")
xx = np.linspace(0, 5, 1000)
plt.plot(xx,[regnew1(xxx) for xxx in xx],"-r",label="True regression function")
plt.xlabel("x")
plt.title('d=1, piecewise constant function')
plt.legend(loc="best")
plt.show()